# **Sewage pAIpes Neural Network Solver**

Implementation details can be found in the main.ipynb section of the report

In [33]:
import torch
from torch import nn 
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
import csv

## Load Data
Load data set from `data/train.csv` and `data/test.csv`

In [34]:
class PipesDataset(Dataset):
    def __init__(self, path: str):
        self.path = path
        curr_dir = os.getcwd()
        data_path = os.path.join(curr_dir, path)
        self.df = pd.read_csv(data_path, dtype=str)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        all = self.df.iloc[idx]

        state = all.iloc[0]
        action = all.iloc[1]

        # Create a list, where each entry in the list is an int
        # the list as a whole represents the state of the board
        state_int_list = [int(x) for x in state]
        state_tensor = torch.tensor(state_int_list)

        action_int_list = [int(x) for x in action]
        action_tensor = torch.tensor(action_int_list)

        return (state_tensor, action_tensor)


train_pipes = DataLoader(PipesDataset("data/train.csv"), batch_size=64, shuffle=True)
test_pipes = DataLoader(PipesDataset("data/test.csv"), batch_size=64, shuffle=True)
train_features, train_labels = next(iter(train_pipes))
test_features, test_labels = next(iter(test_pipes))

## Prepare for Training

- Define network layers
- Set device
- Set hyper-parameters
    - Learning rate
    - Batch size
    - Loss function
    - Optimizer function
- Define test/train loop

In [ ]:
class PipesPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        return self.layers(x)

device = (
    torch.accelerator.current_accelerator().type
    if torch.accelerator.is_available()
    else "cpu"
)
print(f"Using {device} device")

n = 4

learning_rate = 1e-3
batch_size = 64

# Initialize the loss function
loss_fn = nn.BCEWithLogitsLoss()

model = PipesPredictor(input_size=4*(n**2), hidden_size=128, output_size=n**2).to(device)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device).float(), y.to(device).float()
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # if batch % 100 == 0:
        #     loss, current = loss.item(), batch * batch_size + len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0
    total_correct = 0  # total correct label predictions
    total_labels = 0   # total number of labels across all samples

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).float(), y.to(device).float()

            logits = model(X)
            loss = loss_fn(logits, y)
            test_loss += loss.item()

            # Convert logits to probabilities and then to a 0/1 mask.
            probs = torch.sigmoid(logits)
            predicted_mask = (probs >= 0.5).float()

            # Count all correct label predictions (element-wise comparison)
            total_correct += (predicted_mask == y).float().sum().item()
            total_labels += y.numel()  # count of all individual labels

    avg_loss = test_loss / num_batches
    # Calculate average per-label accuracy as a percentage.
    label_accuracy = (total_correct / total_labels) * 100
    print(f"Test Avg loss: {avg_loss:.4f}, Average Label Accuracy: {label_accuracy:.2f}%")

## Training

Train neural network and save it as `model.pth`

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_pipes, model, loss_fn, optimizer)
    test_loop(test_pipes, model, loss_fn)
print("Done!")
model_file_name = "model.pth"
curr_dir = os.getcwd()
model_file_path = os.path.join(curr_dir, model_file_name)
torch.save(model.state_dict(), model_file_path)

## FINE-TUNE ONLY: Train From Existing Model Weight

Load an existing model weight from `model.pth` and train from there. Only do this when you are fine-tuning an existing model.

In [ ]:
model_file_name = "model.pth"
curr_dir = os.getcwd()
model_file_path = os.path.join(curr_dir, model_file_name)
model.load_state_dict(torch.load(model_file_path, weights_only=True, map_location=device))

epochs = 4
learning_rate = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_pipes, model, loss_fn, optimizer)
    test_loop(test_pipes, model, loss_fn)
print("Done!")
model_file_name = "model.pth"
curr_dir = os.getcwd()
model_file_path = os.path.join(curr_dir, model_file_name)
torch.save(model.state_dict(), model_file_path)

## Solve Puzzles Using Model

Solves all puzzles from `puzzles.csv` by repeatedly turning the pipe with the highest label, disallowing making the same move on the same state twice, preventing loops.

We keep track of **outliers**, which are puzzles that take an unusually large amount of moves to solve. Outliers are currently defined as requiring >70 moves.

In [ ]:
def pick_move(state, visited):
    # convert the state to integers
    state_int_list = [int(x) for x in state]
    # convert the state to a tensor
    state_tensor = torch.tensor(state_int_list).to(device).float()
    # get the predicted move from the neural network
    prob = model(state_tensor)
    results = torch.topk(prob, 16).indices

    # Initialize the visited state if we haven't seen it before
    if state not in visited:
        visited[state] = set()

    # Try each of the top moves
    result = None
    for r in results:
        move = int(r)
        # Skip if we've already tried this move for this state
        if not move in visited[state]:
            result = move
            break

    if result is None:
        raise Exception("No valid moves available")
    return result


def pipe_rotate_binary(board: str, pipe: int):
    """
    Takes a binary representation of a board of pipes as a string, and a pipe to rotate. Outputs a binary representation of the board after rotating the pipe.

    :params pipe: The pipe to rotate
    :params board: Binary representation of the board as a string

    """
    # each pipe has 4 values associated to it, so pipe n starts at index 4 * n
    start_index = 4 * pipe
    up = board[start_index]
    right = board[start_index + 1]
    down = board[start_index + 2]
    left = board[start_index + 3]

    # rotate clockwise
    new_board = (
        board[:start_index] + left + up + right + down + board[start_index + 4 :]
    )

    return new_board


def get_bad_pipes(state, goal):
    bad = []
    # Compare each 4-bit chunk (representing a pipe)
    for i in range(0, len(state), 4):
        if state[i : i + 4] != goal[i : i + 4]:
            bad.append(i // 4)
    return bad

model.eval()

# Load puzzles from `puzzles.csv`
initials: list[str] = []
goals: list[str] = []
prev_moves: list[int] = []
puzzle_path = os.path.join(curr_dir, "data/puzzles.csv")
with open(puzzle_path, newline="") as csvfile:
    reader = csv.reader(csvfile)
    # skip the header
    next(reader)
    for row in reader:
        initials.append(row[0])
        goals.append(row[1])
        prev_moves.append(int(row[2]))

outlier_threshold = 70

corrects = 0
bad_pipes_when_incorrect = []
all_moves = []
extra_moves = []
bad_pipes_on_moves = []
normal_initials = []
normal_goals = []

outlier_corrects = 0
outlier_bad_pipes_when_incorrect = []
outlier_all_moves = []
outlier_extra_moves = []
outlier_bad_pipes_on_moves = []

outlier_initials = []
outlier_goals = []

for i in range(len(initials)):
    initial = initials[i]
    goal = goals[i]
    visited = {}

    state = initial
    visited[initial] = set()
    cur_puzzle_moves = 0
    cur_puzzle_bad_pipes_when_incorrect = []
    cur_puzzle_corrects = 0

    cur_puzzle_bad_pipes_on_moves = []


    while state != goal:
        move = pick_move(state, visited)
        bad_pipes = get_bad_pipes(state, goal)
        cur_puzzle_bad_pipes_on_moves.append(len(bad_pipes))
        if move in bad_pipes:
            cur_puzzle_corrects += 1
        else:
            cur_puzzle_bad_pipes_when_incorrect.append(len(bad_pipes))
        visited[state].add(move)
        state = pipe_rotate_binary(state, move)
        cur_puzzle_moves += 1

    if len(all_moves)+len(outlier_all_moves) > 0:
        percentage_outliers = round(len(outlier_all_moves) / (len(all_moves) + len(outlier_all_moves)) * 100, 2)
    else:
        percentage_outliers = 0
    print(f"Solution {i+1}: {cur_puzzle_moves} moves. {percentage_outliers}% outliers")

    if cur_puzzle_moves - prev_moves[i] < 0:
        raise Exception(
            f"Solution {i+1} has {cur_puzzle_moves} moves, but the minimum number of moves is {prev_moves[i]}"
        )

    if cur_puzzle_moves < outlier_threshold:
        corrects += cur_puzzle_corrects

        all_moves.append(cur_puzzle_moves)
        extra_moves.append(cur_puzzle_moves - prev_moves[i])

        bad_pipes_on_moves.extend(cur_puzzle_bad_pipes_on_moves)
        bad_pipes_when_incorrect.extend(cur_puzzle_bad_pipes_when_incorrect)

        normal_initials.append(initial)
        normal_goals.append(goal)
    else:
        outlier_corrects += cur_puzzle_corrects

        outlier_all_moves.append(cur_puzzle_moves)
        outlier_extra_moves.append(cur_puzzle_moves - prev_moves[i])

        outlier_bad_pipes_when_incorrect.extend(cur_puzzle_bad_pipes_when_incorrect)
        outlier_bad_pipes_on_moves.extend(cur_puzzle_bad_pipes_on_moves)

        outlier_initials.append(initial)
        outlier_goals.append(goal)

## Solving Statistics

Displays statistics on the solving process of all puzzles, this separates statistics between normal puzzles and outlier puzzles for easier comparison. For both normal and outliers, we display the **accuracy**, which is defined as the percentage of moves it makes that were correct.

we also display average and median of:
- Moves (number of moves it took to solve)
- Extra moves (number of extra moves it took to solve compared to the optimal minimal moves required)
- Bad pipes after moves (number of pipes that are in incorrect orientations after every move)
    - Helps us understand what kind of states the model spends most of its time on
- Bad pipes when incorrect (number of pipes that are in incorrect orientations when model makes a mistake)
    - Helps us understand what kind of states it's struggling on
- Initial bad pipes (number of of pipes that are in incorrect orientations in the beginning)
    - Helps us understand how the puzzles are scrambled initially

### Graphs on Time Spent on Each State
We plot bad pipes after every move for both normal and outlier puzzles to observe its distribution. Despite average/median being similar,
the distributions are wildly different.

**Normal** bad pipe frequency is skewed towards lower values, suggesting that it's spending most of its time close to completion, which is to be expected as the puzzle gets harder for the network the closer it is to completion.

**Outlier** bad pipe frequency is a normal distribution centered roughly in the middle, suggesting that it's not making significant progress and spending most of its time with roughly half the pipes at incorrect orientations.

### Graphs for Mistakes on Each State
We plot number of bad pipes when it makes it mistake on both normal and outliers. Both are distributed normally.

**Normal** mistakes are centered towards 3 bad pipes, although the frequency of mistakes is so low that it's not a real issue.

**Outlier** mistakes are centered at 6 and 7, suggesting that it is having a hard time getting out that state where roughly half the pipes are incorrect.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print("--------------------------------")
# print percentage that are outliers
percentage_outliers = round(len(outlier_all_moves) / (len(all_moves) + len(outlier_all_moves)) * 100, 2)
print(f"{percentage_outliers}% outliers")
print()
print("--------------------------------")
print("NON-OUTLIER RESULTS")

print(f"Accuracy: {corrects / sum(all_moves)}")

print()
print("MOVES STATISTICS")
print(f"Average moves: {sum(all_moves) / len(all_moves)}")
print(f"Median moves: {sorted(all_moves)[len(all_moves) // 2]}")
print(f"Average extra moves: {sum(extra_moves) / len(extra_moves)}")
print(f"Median extra moves: {sorted(extra_moves)[len(extra_moves) // 2]}")

print()

print("BAD PIPES STATISTICS")
print(f"Average bad pipes after moves: {sum(bad_pipes_on_moves) / len(bad_pipes_on_moves)}")
print(f"Median bad pipes after moves: {sorted(bad_pipes_on_moves)[len(bad_pipes_on_moves) // 2]}")
if len(bad_pipes_when_incorrect) > 0:
    print(
        f"Average bad pipes when incorrect: {sum(bad_pipes_when_incorrect) / len(bad_pipes_when_incorrect)}"
    )
    print(f"Median bad pipes when incorrect: {sorted(bad_pipes_when_incorrect)[len(bad_pipes_when_incorrect) // 2]}")
else:
    print("No incorrect moves on non-outliers!!!")
initial_bad_pipes = [len(get_bad_pipes(normal_initials[i], normal_goals[i])) for i in range(len(normal_initials))]
print(f"Average initial bad pipes: {sum(initial_bad_pipes) / len(initial_bad_pipes)}")
print(f"Median initial bad pipes: {sorted(initial_bad_pipes)[len(initial_bad_pipes) // 2]}")


print()
print("--------------------------------")
print("OUTLIER RESULTS")

print()

print("OUTLIER STATISTICS")
print(f"{len(outlier_all_moves)} Outliers: {outlier_all_moves}")
print(f"Max outlier: {max(outlier_all_moves)}")

print()

print(f"Accuracy: {outlier_corrects / sum(outlier_all_moves)}")

print()

print("MOVES STATISTICS")
print(f"Average moves: {sum(outlier_all_moves) / len(outlier_all_moves)}")
print(f"Median moves: {sorted(outlier_all_moves)[len(outlier_all_moves) // 2]}")
print(f"Average extra moves: {sum(outlier_extra_moves) / len(outlier_extra_moves)}")
print(f"Median extra moves: {sorted(outlier_extra_moves)[len(outlier_extra_moves) // 2]}")

print()

print("BAD PIPES STATISTICS")
print(f"Average bad pipes after moves: {sum(outlier_bad_pipes_on_moves) / len(outlier_bad_pipes_on_moves)}")
print(f"Median bad pipes after moves: {sorted(outlier_bad_pipes_on_moves)[len(outlier_bad_pipes_on_moves) // 2]}")
if len(outlier_bad_pipes_when_incorrect) > 0:
    print(
        f"Average bad pipes when incorrect: {sum(outlier_bad_pipes_when_incorrect) / len(outlier_bad_pipes_when_incorrect)}"
    )
    print(f"Median bad pipes when incorrect: {sorted(outlier_bad_pipes_when_incorrect)[len(outlier_bad_pipes_when_incorrect) // 2]}")
else:
    print("No incorrect moves on outliers!!!")

initial_bad_pipes = [len(get_bad_pipes(outlier_initials[i], outlier_goals[i])) for i in range(len(outlier_initials))]
print(f"Average initial bad pipes: {sum(initial_bad_pipes) / len(initial_bad_pipes)}")
print(f"Median initial bad pipes: {sorted(initial_bad_pipes)[len(initial_bad_pipes) // 2]}")


array1 = bad_pipes_on_moves
array2 = outlier_bad_pipes_on_moves

array3 = bad_pipes_when_incorrect
array4 = outlier_bad_pipes_when_incorrect

# Bin edges — combine arrays in each pair for consistent bins
bins1 = np.histogram_bin_edges(np.concatenate([array1, array2]), bins='auto')
bins2 = np.histogram_bin_edges(np.concatenate([array3, array4]), bins='auto')

# Plot both comparison histograms
fig, axs = plt.subplots(2, 1, figsize=(10, 12))

# First comparison graph
axs[0].hist(array1, bins=bins1, alpha=0.6, edgecolor='black', label='Normal')
axs[0].hist(array2, bins=bins1, alpha=0.6, edgecolor='black', label='Outliers')
axs[0].set_title("Time Spent on Each State")
axs[0].set_xlabel("Number of Bad Pipes")
axs[0].set_ylabel("Frequency")
axs[0].legend()
axs[0].tick_params(axis='x', rotation=45)

# Second comparison graph
axs[1].hist(array3, bins=bins2, alpha=0.6, edgecolor='black', label='Normal')
axs[1].hist(array4, bins=bins2, alpha=0.6, edgecolor='black', label='Outliers')
axs[1].set_title("Mistakes on Each State")
axs[1].set_xlabel("Number of Bad Pipes")
axs[1].set_ylabel("Frequency")
axs[1].legend()
axs[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()



## AFTER-FINE-TUNING: Re-solving Outlier 

Re-solves outlier puzzles. This cell is meant to be ran after the model has been fine-tuned on the augmented outlier data.

To fine-tune the model on augmented data
1. Run the below cell to output the outliers.
2. Generate new data with `dl_data.py`
3. Run `dl_data.py --aug` to augment the outliers
4. Run all cells above FINE-TUNE-ONLY cell except for Training
5. Run the FINE_TUNE-ONLY cell

In [ ]:
model_file_name = "model.pth"
model.load_state_dict(torch.load(model_file_path, weights_only=True))
model.eval()

initials: list[str] = []
goals: list[str] = []
prev_moves: list[int] = []
extra_moves: list[int] = []
still_outliers: list[tuple[str, str, str, str]] = []


puzzle_path = os.path.join(curr_dir, "data/outliers.csv")
with open(puzzle_path, newline="") as csvfile:
    reader = csv.reader(csvfile)
    # skip the header
    next(reader)
    for row in reader:
        initials.append(row[0])
        goals.append(row[1])
        prev_moves.append(int(row[2]))
        extra_moves.append(int(row[3]))

outlier_new_corrects = 0
outlier_new_all_moves = []

for i in range(len(initials)):
    initial = initials[i]
    goal = goals[i]
    visited = {}

    state = initial
    visited[initial] = set()
    cur_puzzle_moves = 0
    cur_puzzle_corrects = 0

    while state != goal:
        move = pick_move(state, visited)
        bad_pipes = get_bad_pipes(state, goal)
        if move in bad_pipes:
            outlier_new_corrects += 1
        visited[state].add(move)
        state = pipe_rotate_binary(state, move)
        cur_puzzle_moves += 1

    print(f"Solution {i+1}: {cur_puzzle_moves} moves")

    if cur_puzzle_moves > 70:
        still_outliers.append((str(initial), str(goal), str(cur_puzzle_moves), str(cur_puzzle_moves - (prev_moves[i]-extra_moves[i]))))
    outlier_new_all_moves.append(cur_puzzle_moves)
    outlier_new_corrects += cur_puzzle_corrects

diffs = [outlier_new_all_moves[i] - prev_moves[i] for i in range(len(outlier_new_all_moves))]

## Output Outliers

Output outliers to `data/outliers.csv`. Storing initial state, goal state, moves it took to solve, and extra moves

In [31]:
# Outputting outliers to data/outliers.csv
with open("data/outliers.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["initial_state", "solution_state", "moves", "extra_moves"])
    for i in range(len(outlier_goals)):
        writer.writerow([outlier_initials[i], outlier_goals[i], outlier_all_moves[i], outlier_extra_moves[i]])

## Statistics on New Outlier Performance 

Displays statistics on its new performance on outliers

In [ ]:
print(f"Accuracy: {outlier_new_corrects / sum(outlier_new_all_moves)}")
print(f"Average moves: {sum(outlier_new_all_moves) / len(outlier_new_all_moves)}")
print(f"Median moves: {sorted(outlier_new_all_moves)[len(outlier_new_all_moves) // 2]}")
print(f"Diff: {diffs}")
print(f"Average diff: {sum(diffs) / len(diffs)}")
print(f"Median diff: {sorted(diffs)[len(diffs) // 2]}")

print(f"Solved {len(outlier_new_all_moves)-len(still_outliers)}/{len(outlier_new_all_moves)} outliers")

# graph the diffs
plt.bar(range(len(diffs)), diffs)

# Optional: Add labels and title
plt.xlabel('Index')
plt.ylabel('Difference')
plt.title('Move Difference of Outlier Puzzles')

## Append Remaining Outliers

Appends previous outliers that are still outliers to `outliers.csv`. This cell is meant to be ran after the newly fine-tuned model solved all previous outliers and after we outputted the new outliers found with the newly fine-tuned model.

In [32]:
with open("data/outliers.csv", "a") as f:
    for still_outlier in still_outliers:
        f.write(f"{still_outlier[0]},{still_outlier[1]},{still_outlier[2]},{still_outlier[3]}\n")